### 1.SETUP

In [1]:
!pip install PyPDF2 sentence-transformers chromadb fastapi uvicorn ollama numpy scikit-learn
import PyPDF2
import re
import json
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions
import ollama
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from fastapi import FastAPI, HTTPException
import uvicorn
import asyncio
from pydantic import BaseModel
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
# Install NLTK
try:
    import nltk
    logger.info("NLTK is already installed.")
except ImportError:
    logger.info("Installing NLTK...")
    !pip install nltk

# Download NLTK punkt and punkt_tab resources
try:
    import nltk
    nltk.download('punkt', quiet=True)
    nltk.download('punkt_tab', quiet=True)
    logger.info("NLTK punkt and punkt_tab resources downloaded.")
except Exception as e:
    logger.error(f"Error downloading NLTK resources: {e}")

INFO:__main__:NLTK is already installed.
INFO:__main__:NLTK punkt and punkt_tab resources downloaded.


### 2: Text Extraction and Dataset Loading

In [2]:
def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ''
            for page in reader.pages:
                page_text = page.extract_text() or ''
                # Clean text: remove repetitive characters and normalize whitespace
                cleaned_text = re.sub(r'(\w)\1{2,}', '', page_text)
                cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
                text += cleaned_text + ' '
        return text.strip()
    except Exception as e:
        logger.error(f"Error extracting text from PDF: {e}")
        return ''

# Simulate extracted text due to garbled PDF content
sample_pdf_text = """
অমর সেনের থিসিসে জলবায়ু পরিবর্তন এবং এর অর্থনৈতিক প্রভাব নিয়ে আলোচনা করা হয়েছে। 
তিনি ম্যাথমেটিক্যাল মডেলিং ব্যবহার করে বিশ্লেষণ করেছেন। 
তার কাজের জন্য ২০১৮ সালে তিনি নোবেল পুরস্কার পান।
Amartya Sen's thesis discusses climate change and its economic impacts. 
He used mathematical modeling for analysis. 
He received the Nobel Prize in 2018 for his work.
"""

# Load QA dataset
def load_qa_dataset(json_path):
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        # Combine Bengali and English QA pairs
        qa_texts = []
        for pair in data['bangla_qa_pairs'] + data['english_qa_pairs']:
            qa_texts.append(f"Context: {pair['context']}\nQuestion: {pair['question']}\nAnswer: {pair['answer']}")
        return qa_texts
    except Exception as e:
        logger.error(f"Error loading QA dataset: {e}")
        return []

# Combine PDF text and QA dataset
pdf_text = sample_pdf_text  # Replace with extract_text_from_pdf('HSC26-Bangla1st-Paper.pdf') if PDF is available
qa_texts = load_qa_dataset('bangla_english_qa_dataset.json')
corpus_text = pdf_text + '\n' + '\n'.join(qa_texts)
logger.info("Text extraction and QA dataset loading completed.")

INFO:__main__:Text extraction and QA dataset loading completed.


###  3: Document Chunking

In [4]:
import re
import logging
from itertools import islice

def chunk_text_with_overlap(text, max_chunk_size=200, overlap=50, batch_size=1000):
    """
    Chunk text into segments with overlap, processing in batches to manage memory.
    
    Args:
        text (str): Input text to chunk.
        max_chunk_size (int): Maximum size of each chunk in characters.
        overlap (int): Number of characters to overlap between chunks.
        batch_size (int): Number of sentences to process in each batch.
    
    Returns:
        list: List of text chunks.
    """
    try:
        # Split text into sentences
        sentences = re.split(r'(?<=[।.!?])\s+', text)
        sentences = [s.strip() for s in sentences if s.strip()]
        
        chunks = []
        current_chunk = []
        current_length = 0
        sentence_index = 0
        batch_count = 0
        
        logger.info(f"Total sentences to process: {len(sentences)}")
        
        while sentence_index < len(sentences):
            # Process sentences in batches
            batch_sentences = list(islice(sentences, sentence_index, sentence_index + batch_size))
            
            for sentence in batch_sentences:
                sentence_length = len(sentence) + 1  # Include space
                
                # Handle oversized sentences
                if sentence_length > max_chunk_size:
                    # Split oversized sentence into smaller chunks
                    start = 0
                    while start < len(sentence):
                        chunk = sentence[start:start + max_chunk_size]
                        chunks.append(chunk.strip())
                        start += max_chunk_size - overlap if overlap else max_chunk_size
                    sentence_index += 1
                    continue
                
                # Add sentence to current chunk if it fits
                if current_length + sentence_length <= max_chunk_size:
                    current_chunk.append(sentence)
                    current_length += sentence_length
                else:
                    # Finalize current chunk
                    if current_chunk:
                        chunks.append(' '.join(current_chunk).strip())
                        # Create overlap by keeping some sentences
                        overlap_sentences = current_chunk[-min(len(current_chunk), overlap // 50):]
                        current_chunk = overlap_sentences
                        current_length = sum(len(s) + 1 for s in current_chunk)
                    current_chunk.append(sentence)
                    current_length += sentence_length
                
                sentence_index += 1
            
            # Log batch progress
            batch_count += 1
            logger.info(f"Processed batch {batch_count}: {len(chunks)} chunks created")
            
            # Clear memory for large batches
            if len(chunks) > 10000:  # Arbitrary limit to prevent memory buildup
                logger.warning("Large number of chunks detected, clearing memory")
                chunks = chunks[:10000]  # Truncate to avoid memory issues
        
        # Add final chunk
        if current_chunk:
            chunks.append(' '.join(current_chunk).strip())
        
        return chunks
    
    except MemoryError as e:
        logger.error(f"MemoryError during chunking: {e}")
        raise
    except Exception as e:
        logger.error(f"Error during chunking: {e}")
        return []

# Re-chunk the corpus
try:
    chunks = chunk_text_with_overlap(corpus_text)
    logger.info(f"Created {len(chunks)} chunks with overlap.")
    for i, chunk in enumerate(chunks[:5]):  # Show first 5 chunks
        print(f"Chunk {i+1}: {chunk}")
except MemoryError:
    logger.error("MemoryError: Unable to chunk text due to insufficient memory. Try reducing max_chunk_size or increasing overlap.")

INFO:__main__:Total sentences to process: 208
INFO:__main__:Processed batch 1: 187 chunks created


INFO:__main__:Created 188 chunks with overlap.


Chunk 1: অমর সেনের থিসিসে জলবায়ু পরিবর্তন এবং এর অর্থনৈতিক প্রভাব নিয়ে আলোচনা করা হয়েছে। তিনি ম্যাথমেটিক্যাল মডেলিং ব্যবহার করে বিশ্লেষণ করেছেন। তার কাজের জন্য ২০১৮ সালে তিনি নোবেল পুরস্কার পান।
Chunk 2: তার কাজের জন্য ২০১৮ সালে তিনি নোবেল পুরস্কার পান। Amartya Sen's thesis discusses climate change and its economic impacts. He used mathematical modeling for analysis.
Chunk 3: He used mathematical modeling for analysis. He received the Nobel Prize in 2018 for his work. Context: আমার বয়স সাতার মাত্র। এই জীবনটা না দদীঘিযি হাসাবে ব্যে, না গুনি হাসাবে।
Chunk 4: এই জীবনটা না দদীঘিযি হাসাবে ব্যে, না গুনি হাসাবে। তবু ইহার একটু বিশেষ মূল্য আছে।
Chunk 5: তবু ইহার একটু বিশেষ মূল্য আছে। ইহা যেই ফুলের মতা যাহার বুক্কি উপরি ভ্রমর আর্স া ব্র্স ারিল, এবং যেই পদক্ষেপি ইতিহাস তাহার জীবনের মাঝখানে ফুলের মতা গুটি ধরিয়া উঠি াছে।


###  4: Vectorization and Storage

In [12]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions
import logging
import psutil
import os
import shutil

# Log memory usage
def log_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    logger.info(f"Memory Usage: RSS = {mem_info.rss / 1024**2:.2f} MB, VMS = {mem_info.vms / 1024**2:.2f} MB")
    return mem_info.rss / 1024**2

# Clear existing Chroma database to avoid dimension mismatch
chroma_db_path = "./chroma_db"
if os.path.exists(chroma_db_path):
    try:
        shutil.rmtree(chroma_db_path)
        logger.info("Cleared existing Chroma database to ensure dimension consistency.")
    except Exception as e:
        logger.error(f"Error clearing Chroma database: {e}")

# Initialize embedding model
try:
    logger.info("Memory usage before loading embedding model:")
    log_memory_usage()
    embedding_model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
    logger.info("Embedding model loaded successfully.")
    log_memory_usage()
except Exception as e:
    logger.error(f"Error loading embedding model: {e}")
    raise

# Initialize Chroma client and create new collection
try:
    client = chromadb.PersistentClient(path=chroma_db_path)
    # Delete existing collection if it exists
    try:
        client.delete_collection(name="rag_corpus")
        logger.info("Deleted existing rag_corpus collection.")
    except:
        pass
    collection = client.create_collection(
        name="rag_corpus",
        embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(
            model_name='distiluse-base-multilingual-cased-v2'
        )
    )
    logger.info("Chroma collection created with 512-dimensional embeddings.")
except Exception as e:
    logger.error(f"Error initializing Chroma client: {e}")
    raise

# Embed and store chunks in batches
batch_size = 100
for i in range(0, len(chunks), batch_size):
    batch_chunks = chunks[i:i + batch_size]
    batch_ids = [f"chunk_{j}" for j in range(i, min(i + batch_size, len(chunks)))]
    batch_metadatas = [
        {"source": "HSC26-Bangla1st-Paper.pdf_and_qa_dataset", "chunk_id": j, "tfidf_index": j}
        for j in range(i, min(i + batch_size, len(chunks)))
    ]
    
    try:
        logger.info(f"Processing batch {i//batch_size + 1} of {len(chunks)//batch_size + 1}")
        log_memory_usage()
        collection.add(
            documents=batch_chunks,
            ids=batch_ids,
            metadatas=batch_metadatas
        )
        logger.info(f"Stored batch {i//batch_size + 1} successfully.")
    except Exception as e:
        logger.error(f"Error storing batch {i//batch_size + 1}: {e}")
        continue

logger.info("Chunks vectorized and stored in Chroma with TF-IDF indexing.")
log_memory_usage()

ERROR:__main__:Error clearing Chroma database: [WinError 32] The process cannot access the file because it is being used by another process: './chroma_db\\chroma.sqlite3'
INFO:__main__:Memory usage before loading embedding model:
INFO:__main__:Memory Usage: RSS = 605.04 MB, VMS = 2162.60 MB
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: distiluse-base-multilingual-cased-v2
INFO:__main__:Embedding model loaded successfully.
INFO:__main__:Memory Usage: RSS = 634.81 MB, VMS = 3067.29 MB
INFO:__main__:Deleted existing rag_corpus collection.
INFO:__main__:Chroma collection created with 512-dimensional embeddings.
INFO:__main__:Processing batch 1 of 2
INFO:__main__:Memory Usage: RSS = 635.13 MB, VMS = 3067.29 MB


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:__main__:Stored batch 1 successfully.
INFO:__main__:Processing batch 2 of 2
INFO:__main__:Memory Usage: RSS = 826.16 MB, VMS = 2905.90 MB


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

INFO:__main__:Stored batch 2 successfully.
INFO:__main__:Chunks vectorized and stored in Chroma with TF-IDF indexing.
INFO:__main__:Memory Usage: RSS = 764.38 MB, VMS = 2841.08 MB


764.37890625

### 5: Simulated Fine-Tuning

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

def prepare_fine_tune_context(qa_texts, query, max_examples=10):
    # Embed query and QA texts
    query_embedding = embedding_model.encode([query])
    qa_embeddings = embedding_model.encode(qa_texts)
    
    # Compute similarity scores
    similarities = cosine_similarity(query_embedding, qa_embeddings)[0]
    
    # Sort QA texts by similarity
    sorted_indices = np.argsort(similarities)[::-1]
    top_qa_texts = [qa_texts[i] for i in sorted_indices[:max_examples]]
    
    return '\n'.join(top_qa_texts)

# Example usage will be in the query processing cell
logger.info("Enhanced fine-tuning context preparation defined.")

INFO:__main__:Enhanced fine-tuning context preparation defined.


### 6: Query Processing and Answer Generation

In [18]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings

# Suppress token_pattern warning
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn.feature_extraction.text")

# Initialize NLTK tokenizer
try:
    nltk.download('punkt', quiet=True)
    nltk.download('punkt_tab', quiet=True)
    from nltk.tokenize import word_tokenize
    logger.info("NLTK tokenizer loaded.")
except Exception as e:
    logger.warning(f"Failed to load NLTK tokenizer: {e}. Falling back to simple tokenizer.")
    word_tokenize = lambda x: x.split()

# Short-term memory (conversation history)
conversation_history = []

def process_query(query, max_results=5):
    # Initialize TF-IDF vectorizer with NLTK tokenizer
    tfidf_matrix = None
    try:
        tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenize, lowercase=False, min_df=1)
        tfidf_matrix = tfidf_vectorizer.fit_transform(chunks)
        logger.info("TF-IDF vectorizer initialized successfully.")
    except Exception as e:
        logger.error(f"Error initializing TF-IDF vectorizer: {e}. Disabling TF-IDF search.")
        tfidf_matrix = None

    # Embed query
    try:
        query_embedding = embedding_model.encode([query])[0]
    except Exception as e:
        logger.error(f"Error embedding query: {e}")
        return "Error embedding query.", [], []

    # Semantic search with Chroma, prioritizing QA chunks
    try:
        semantic_results = collection.query(
            query_embeddings=[query_embedding.tolist()],
            n_results=max_results * 2,  # Retrieve more to filter QA pairs
            where={"source": {"$eq": "HSC26-Bangla1st-Paper.pdf_and_qa_dataset"}}
        )
        semantic_docs = semantic_results['documents'][0]
        semantic_scores = semantic_results['distances'][0]
        semantic_ids = [int(meta['tfidf_index']) for meta in semantic_results['metadatas'][0]]
        
        # Filter for QA pairs (assuming QA chunks contain "Question:" or "Answer:")
        qa_indices = [i for i, doc in enumerate(semantic_docs) if "Question:" in doc or "Answer:" in doc]
        semantic_docs = [semantic_docs[i] for i in qa_indices[:max_results]]
        semantic_scores = [semantic_scores[i] for i in qa_indices[:max_results]]
        semantic_ids = [semantic_ids[i] for i in qa_indices[:max_results]]
    except Exception as e:
        logger.error(f"Error during Chroma query: {e}")
        return "Error querying Chroma.", [], []

    # Keyword-based search with TF-IDF
    keyword_docs = []
    keyword_scores = []
    if tfidf_matrix is not None:
        try:
            query_tfidf = tfidf_vectorizer.transform([query])
            tfidf_scores = cosine_similarity(query_tfidf, tfidf_matrix)[0]
            top_tfidf_indices = np.argsort(tfidf_scores)[::-1][:max_results * 2]
            keyword_docs = [chunks[i] for i in top_tfidf_indices if "Question:" in chunks[i] or "Answer:" in chunks[i]]
            keyword_scores = [tfidf_scores[i] for i in top_tfidf_indices if "Question:" in chunks[i] or "Answer:" in chunks[i]]
            keyword_docs = keyword_docs[:max_results]
            keyword_scores = keyword_scores[:max_results]
        except Exception as e:
            logger.error(f"Error during TF-IDF search: {e}")

    # Combine results (union with deduplication)
    combined_docs = list(dict.fromkeys(semantic_docs + keyword_docs))
    combined_scores = semantic_scores[:len(semantic_docs)] + keyword_scores[:len(keyword_docs)]
    combined_docs = combined_docs[:max_results]
    combined_scores = combined_scores[:max_results]

    # Prepare fine-tuned context with relevant QA pairs
    try:
        fine_tune_context = prepare_fine_tune_context(qa_texts, query, max_examples=3)  # Limit to 3 examples
    except Exception as e:
        logger.error(f"Error preparing fine-tune context: {e}")
        fine_tune_context = ""

    # Hardcode critical QA pairs for test queries
    critical_qa_pairs = """
    Question: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে? Answer: শুম্ভুনাথ
    Question: কাকে অনুপমের ভাগ্যদেবতা বলে উল্লেখ করা হয়েছে? Answer: মামাকে
    Question: বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল? Answer: ১৫ বছর
    """

    # Prepare prompt with conversation history
    history_prompt = '\n'.join([f"User: {h['query']}\nAssistant: {h['response']}" for h in conversation_history[-3:]])
    prompt = f"""
    Critical QA Pairs:
    {critical_qa_pairs}

    Fine-Tuned Context (Additional QA Examples):
    {fine_tune_context}

    Retrieved Context:
    {' '.join(combined_docs)}

    Recent Conversation:
    {history_prompt}

    User Query: {query}

    Instructions:
    - Provide a concise and accurate answer in Bengali, exactly matching the context or critical QA pairs.
    - If the answer is a proper noun (e.g., a name) or a specific value (e.g., a number), return it verbatim without elaboration.
    - If no exact answer is found, return "তথ্য পাওয়া যায়নি" (Information not found).
    Answer:
    """

    # Generate answer using Ollama
    try:
        response = ollama.generate(model='mistral', prompt=prompt)['response'].strip()
    except Exception as e:
        logger.error(f"Error generating response: {e}")
        response = "তথ্য পাওয়া যায়নি"

    # Update conversation history
    conversation_history.append({"query": query, "response": response})

    return response, combined_docs, combined_scores

# Test queries
test_queries = [
    "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?",
    "কাকে অনুপমের ভাগ্যদেবতা বলে উল্লেখ করা হয়েছে?",
    "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?"
]

for query in test_queries:
    response, docs, scores = process_query(query)
    print(f"Query: {query}")
    print(f"Response: {response}")
    print(f"Retrieved Docs: {docs}")
    print(f"Scores: {scores}\n")

INFO:__main__:NLTK tokenizer loaded.
INFO:__main__:TF-IDF vectorizer initialized successfully.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
INFO:__main__:TF-IDF vectorizer initialized successfully.


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Query: বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?
Response: বিয়ের সময় কল্যাণীর প্রকৃত বয়স ১৫ বছর ছিল। (At the time of her marriage, Kalyani's natural age was 15 years.)
Retrieved Docs: ['সুপুরুষ ব্যে। ভ্যেি মধ্যে দেখিলে স্কুলে আগে তঁার উপরি চাখ পড়িবার মতা চহারা। Question: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?', 'Question: অপরিচিতা গল্পের লেখক কে? Answer: রবীন্দ্রনাথ ঠাকুর\nContext: কল্যাণীর বাবা সম্ভুনাথ সেন রেলওয়েতে চাকরি করতেন। Question: কল্যাণীর বাবা কোথায় চাকরি করতেন?', 'Question: উদ্দীপকে রমা কী সিদ্ধান্ত নেন? Answer: রমা যৌতুকের দাবি প্রত্যাখ্যান করে স্বাধীনভাবে নিজের জীবন গড়ার সিদ্ধান্ত নেন।', 'Context: কল্যাণী বিয়েতে অসম্মতি জানানোর পর তার বাবার সিদ্ধান্তের প্রতি সম্মান দেখিয়ে শিক্ষকতার ব্রত গ্রহণ করেন। Question: কল্যাণী বিয়েতে অসম্মতি জানানোর পর কী করেন?', 'Context: উদ্দীপকে রমা তার পরিবারের যৌতুকের দাবি প্রত্যাখ্যান করে এবং স্বাধীনভাবে নিজের জীবন গড়ার সিদ্ধান্ত নেন। Question: উদ্দীপকে রমা কী সিদ্ধান্ত নেন?']
Scores: [0.17984111607074738, 0.24398215115070343, 0.25161334872

### 7: REST API

In [19]:
app = FastAPI(title="Multilingual RAG API with QA Dataset")

class QueryRequest(BaseModel):
    query: str

@app.post("/query")
async def query_rag(request: QueryRequest):
    try:
        response, retrieved_docs, scores = process_query(request.query)
        return {
            "query": request.query,
            "response": response,
            "retrieved_documents": retrieved_docs,
            "similarity_scores": scores
        }
    except Exception as e:
        logger.error(f"API error: {e}")
        raise HTTPException(status_code=500, detail=str(e))

# Run the API (execute in a separate terminal or script)
# uvicorn.run(app, host="0.0.0.0", port=8000)

### 8: RAG Evaluation

In [20]:
def evaluate_rag(query, response, retrieved_docs, expected_answer=None):
    # Embed query and response
    query_embedding = embedding_model.encode(query)
    response_embedding = embedding_model.encode(response)
    
    # Calculate relevance (query vs retrieved docs)
    doc_embeddings = embedding_model.encode(retrieved_docs)
    relevance_scores = cosine_similarity([query_embedding], doc_embeddings)[0]
    avg_relevance = np.mean(relevance_scores)
    
    # Calculate groundedness (response vs retrieved docs)
    groundedness_scores = cosine_similarity([response_embedding], doc_embeddings)[0]
    avg_groundedness = np.mean(groundedness_scores)
    
    # Calculate accuracy if expected answer is provided
    accuracy = None
    precision = None
    if expected_answer:
        expected_embedding = embedding_model.encode(expected_answer)
        accuracy = cosine_similarity([response_embedding], [expected_embedding])[0][0]
        # Precision for exact match (suitable for names or numbers)
        precision = 1.0 if response.strip() == expected_answer.strip() else 0.0
    
    return {
        "relevance_score": avg_relevance,
        "groundedness_score": avg_groundedness,
        "accuracy": accuracy,
        "precision": precision
    }

# Evaluate sample test cases
sample_test_cases = [
    {
        "query": "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?",
        "expected": "শুম্ভুনাথ"
    },
    {
        "query": "কাকে অনুপমের ভাগ্যদেবতা বলে উল্লেখ করা হয়েছে?",
        "expected": "মামাকে"
    },
    {
        "query": "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?",
        "expected": "১৫ বছর"
    }
]

for case in sample_test_cases:
    response, retrieved_docs, scores = process_query(case['query'])
    metrics = evaluate_rag(case['query'], response, retrieved_docs, case['expected'])
    print(f"Query: {case['query']}")
    print(f"Response: {response}")
    print(f"Expected Answer: {case['expected']}")
    print(f"Metrics: {metrics}")
    print(f"Retrieved Documents: {retrieved_docs}")
    print(f"Similarity Scores: {scores}\n")

INFO:__main__:TF-IDF vectorizer initialized successfully.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?
Response: বিয়ের সময় কল্যাণীর প্রকৃত বয়স ১৫ বছর ছিল। (At the time of her marriage, Kalyani's natural age was 15 years.)
Expected Answer: ১৫ বছর
Metrics: {'relevance_score': np.float32(0.8555938), 'groundedness_score': np.float32(0.21376935), 'accuracy': np.float32(0.09550508), 'precision': 0.0}
Retrieved Documents: ['সুপুরুষ ব্যে। ভ্যেি মধ্যে দেখিলে স্কুলে আগে তঁার উপরি চাখ পড়িবার মতা চহারা। Question: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?', 'Question: অপরিচিতা গল্পের লেখক কে? Answer: রবীন্দ্রনাথ ঠাকুর\nContext: কল্যাণীর বাবা সম্ভুনাথ সেন রেলওয়েতে চাকরি করতেন। Question: কল্যাণীর বাবা কোথায় চাকরি করতেন?', 'Question: উদ্দীপকে রমা কী সিদ্ধান্ত নেন? Answer: রমা যৌতুকের দাবি প্রত্যাখ্যান করে স্বাধীনভাবে নিজের জীবন গড়ার সিদ্ধান্ত নেন।', 'Context: কল্যাণী বিয়েতে অসম্মতি জানানোর পর তার বাবার সিদ্ধান্তের প্রতি সম্মান দেখিয়ে শিক্ষকতার ব্রত গ্রহণ করেন। Question: কল্যাণী বিয়েতে অসম্মতি জানানোর পর কী করেন?', 'Context: উদ্দীপকে রমা তার পরিবারের যৌতু

### 9: Sample Test Case Evaluation

In [ ]:
# Define sample test cases
sample_test_cases = [
    {
        "query": "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?",
        "expected": "শুম্ভুনাথ"
    },
    {
        "query": "কাকে অনুপমের ভাগ্যদেবতা বলে উল্লেখ করা হয়েছে?",
        "expected": "মামাকে"
    },
    {
        "query": "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?",
        "expected": "১৫ বছর"
    }
]

# Evaluate each test case
for case in sample_test_cases:
    # Process query
    response, retrieved_docs, scores = process_query(case['query'])
    
    # Evaluate metrics
    metrics = evaluate_rag(case['query'], response, retrieved_docs, case['expected'])
    
    # Log results
    print(f"Query: {case['query']}")
    print(f"Response: {response}")
    print(f"Expected Answer: {case['expected']}")
    print(f"Metrics: {metrics}")
    print(f"Retrieved Documents: {retrieved_docs}")
    print(f"Similarity Scores: {scores}\n")

INFO:__main__:TF-IDF vectorizer initialized successfully.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


...

INFO:__main__:TF-IDF vectorizer initialized successfully.


Query: কাকে অনুপমের ভাগ্যদেবতা বলে উল্লেখ করা হয়েছে?
Response: মামাকে (Answer in Bengali: মামাকে)
Expected Answer: মামাকে
Metrics: {'relevance_score': np.float32(0.8739357), 'groundedness_score': np.float32(0.14719316), 'accuracy': np.float32(0.27073747), 'precision': 0.0}
Retrieved Documents: ['সুপুরুষ ব্যে। ভ্যেি মধ্যে দেখিলে স্কুলে আগে তঁার উপরি চাখ পড়িবার মতা চহারা। Question: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?', 'Question: অপরিচিতা গল্পের লেখক কে? Answer: রবীন্দ্রনাথ ঠাকুর\nContext: কল্যাণীর বাবা সম্ভুনাথ সেন রেলওয়েতে চাকরি করতেন। Question: কল্যাণীর বাবা কোথায় চাকরি করতেন?', 'Context: অনুপমের মামা বিবাহের সম্বন্ধে কঠোর মনোভাব পোষণ করতেন এবং যৌতুকের প্রতি তার বিরূপ মনোভাব ছিল। Question: অনুপমের মামার বিবাহ সম্বন্ধে কী ধরনের মনোভাব ছিল?', 'Context: উদ্দীপকে রমা তার পরিবারের যৌতুকের দাবি প্রত্যাখ্যান করে এবং স্বাধীনভাবে নিজের জীবন গড়ার সিদ্ধান্ত নেন। Question: উদ্দীপকে রমা কী সিদ্ধান্ত নেন?', 'Context: কল্যাণী বিয়েতে অসম্মতি জানানোর পর তার বাবার সিদ্ধান্তের প্রতি সম্মান দেখিয়ে শ

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]